In [ ]:
import random
import rtsvg
rt = rtsvg.RACETrack()

In [ ]:
_n_, w, h = 20, 600, 400
pts = []
for i in range(_n_):
    x, y = random.randint(20,w-20), random.randint(20,h-20)
    pts.append((x,y))

# Base Image
_svg_ = [f'<svg x="0" y="0" width="{w}" height="{h}">']
_svg_.append('<rect x="0" y="0" width="{w}" height="{h}" fill="#ffffff" />')
for i in range(_n_): _svg_.append(f'<circle cx="{pts[i][0]}" cy="{pts[i][1]}" r="2" stroke="#000000" fill="#000000" />')

# Perpendicular lines
_perps_ = []
for i in range(_n_):
    for j in range(i+1, _n_):
        x, y = (pts[i][0]+pts[j][0])/2, (pts[i][1]+pts[j][1])/2
        dx, dy = rt.unitVector((pts[i],pts[j]))
        pdx, pdy = dy, -dx
        l = max(w,y)
        _perps_.append(f'<line x1="{x+pdx*l}" y1="{y+pdy*l}" x2="{x-pdx*l}" y2="{y-pdy*l}" stroke="#000000" stroke-width="0.1" />')
        _perps_.append(f'<circle cx="{x}" cy="{y}" r="2" fill="none" stroke="#ff0000" stroke-width="0.2" />')

rt.tile([''.join(_svg_) + '</svg>', ''.join(_svg_) + ''.join(_perps_) + '</svg>'])

In [ ]:
# Perpendicular lines
_single_ = []
i  = 0
_single_.append(f'<circle cx="{pts[0][0]}" cy="{pts[0][1]}" r="4" fill="none" stroke="#ff0000" stroke-width="1" />')
for j in range(1, _n_):
    x, y = (pts[i][0]+pts[j][0])/2, (pts[i][1]+pts[j][1])/2
    dx, dy = rt.unitVector((pts[i],pts[j]))
    pdx, pdy = dy, -dx
    l = max(w,y)
    _single_.append(f'<line x1="{x+pdx*l}" y1="{y+pdy*l}" x2="{x-pdx*l}" y2="{y-pdy*l}" stroke="#000000" stroke-width="0.1" />')
    _single_.append(f'<circle cx="{x}" cy="{y}" r="2" fill="none" stroke="#ff0000" stroke-width="0.2" />')
rt.tile([''.join(_svg_) + ''.join(_single_) + '</svg>'])